In [ ]:
"""Create plots of some results."""
# pylint: disable=import-error,redefined-outer-name

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.ticker import PercentFormatter

## All classifiers performance comparison

In [ ]:
file_path = Path.home() / "downloads" / "temp" / "all_metrics - Pivot Table 1.csv"

In [ ]:
def create_metrics_df(file_path: Path) -> pd.DataFrame:
    """Create a dataframe from the csv file."""
    metrics_df = pd.read_csv(file_path)

    # Fill missing values in the 'classifier' column using forward fill method
    metrics_df["classifier"] = metrics_df["classifier"].fillna(method="ffill")

    # Rename columns to match the desired format
    metrics_df.rename(
        columns={
            "classifier": "Classifier",
            "metric": "Metric",
            "AVERAGE of value": "Average",
            "STDEV of value": "Std",
        },
        inplace=True,
    )

In [ ]:
def plot_classifiers_performance(metrics_df: pd.DataFrame) -> None:
    """Plot the performance of multiple classifiers."""
    # Set the figure size
    plt.figure(figsize=(10, 6))

    # Create a bar plot without error bars
    barplot = sns.barplot(
        data=metrics_df, x="Classifier", y="Average", hue="Metric", errorbar=None
    )

    # Get the x and y coordinates of the bars
    x_coords = []
    y_coords = []
    for rect in barplot.patches:
        x_coords.append(rect.get_x() + rect.get_width() / 2)
        y_coords.append(rect.get_height())

    # Calculate the number of metrics and classifiers to determine the positions of the error bars
    num_metrics = metrics_df["Metric"].nunique()
    num_classifiers = metrics_df["Classifier"].nunique()

    # Add the error bars
    for i in range(num_classifiers):
        for j in range(num_metrics):
            barplot.errorbar(
                x_coords[i * num_metrics + j],
                y_coords[i * num_metrics + j],
                yerr=metrics_df["Std"][i * num_metrics + j],
                color="black",
                capsize=3,
                fmt="none",
            )

    # Set the y-axis limits center the value distribution
    plt.ylim(min(y_coords) - 0.025, min(max(y_coords) + 0.025, 1))  # type: ignore

    # Scale the y-axis to percentage
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))

    plt.title("Classifier Performance")
    plt.ylabel("Performance")

    # Move the legend outside the plot
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

    plt.show()